In [2]:
from pathlib import Path
import pandas as pd

In [82]:
CLASSES = ['Healthy', 'OCDegen 2', 'OCDegen 1']
input_stages = {'0': 'training', '1': 'validation_nodiag', '2': 'validation_noxai'}
BASEPATH = Path('/Users/djohnson/sciebo/1. Research/1. HCXAI/1. Projects/evalxai_studies/example_validation_study')


id = '5ae96d1878b0060001e69b8a'

resfile1 = BASEPATH / input_stages['1'] / 'results' / f'results_{input_stages["1"]}_{id}.csv'
resfile2 = BASEPATH / input_stages['2'] / 'results' / f'results_{input_stages["2"]}_{id}.csv'

In [83]:
df1 = pd.read_csv(resfile1)
df2 = pd.read_csv(resfile2)


In [84]:
# df.loc[8].updated - df.loc[8].accessed

In [85]:
part_acc = (df1['true'] == df1['select']).astype(int).mean()
model_acc = (df1['true'] == df1['pred']).astype(int).mean()

part_model = (df1['pred'] == df1['select']).astype(int).mean()

print(f'Model Acc: {model_acc}')
print(f'Part Acc: {part_acc}')
print(f'Part matches model: {part_model}')

Model Acc: 0.7
Part Acc: 0.6
Part matches model: 0.4


In [86]:
def preprocess_results(df):
   df['part_correct'] = (df['true'] == df['select']).astype(int)

   return df

def get_results(df, stage):
    
    acc = df['part_correct'].mean()
    dec_time = (df['updated'] - df['accessed_updated']).mean()

    # stage one only has an accuracy value since there is no AI and stage 0 is just for sanity checking
    if stage <= 1:
       return acc, dec_time
    
    agree_cond = df['pred'] == df['select']
    disagree_cond = df['pred'] != df['select']

    correct_preds = (df['pred'] == df['true'])
    incorrect_preds = (df['pred'] != df['true'])  

    # overreliance
    or_cond = agree_cond & incorrect_preds
    overreliance = (or_cond).astype(int).sum() / incorrect_preds.astype(int).sum()

    ur_cond = disagree_cond & correct_preds
    underreliance = (ur_cond).astype(int).sum() / correct_preds.astype(int).sum()

    # agreement
    reliance = (agree_cond).astype(int).sum() / len(df)
    
    # disagreement
    disagree = (disagree_cond).astype(int).sum() / len(df)

    # healthy distrust (prev. approp. reliance)
    hdt_cond = disagree_cond & incorrect_preds
    h_distrust = (hdt_cond).astype(int).sum() / (incorrect_preds).astype(int).sum()

    # health trust
    ht_cond = agree_cond & correct_preds
    h_trust = (ht_cond).astype(int).sum() / (correct_preds).astype(int).sum()

    return acc, reliance, overreliance, underreliance, disagree, h_distrust, h_trust, dec_time


In [87]:
get_results(preprocess_results(df1), 2)

(0.6, 0.4, 0.16666666666666666, 0.5, 0.6, 0.8333333333333334, 0.5, 5710.375)

In [88]:
get_results(preprocess_results(df2), 2)

(0.65,
 0.75,
 0.6666666666666666,
 0.21428571428571427,
 0.25,
 0.3333333333333333,
 0.7857142857142857,
 3165.9)